# Analysis of results from Buchi automata experiments
First, let us import some essential packages


In [9]:
import datetime
import pandas as pd
import re as re
import tabulate as tab
import math

import evallib as el               # this contains auxiliary functionality for evaluation of experiments

And we define other useful auxiliary functions.

In [10]:
# Connect a DF with results to DF with classification of inputs
def connect_with_classification(df, clas_file):
    df_clas = el.read_file(clas_file)
    df = pd.merge(df, df_clas, on='name')
    return df


# prints results of classification
def print_classification(df):
    df_empty = df[df['empty'] == 1]
    df_deterministic = df[df['deterministic'] == 1]
    df_deterministic_weak = df[(df['deterministic'] == 1) & (df['weak'] == 1)]
    df_inherently_weak = df[df['inherently weak'] == 1]
    df_semi_deterministic = df[df['semi deterministic'] == 1]
    df_terminal = df[df['terminal'] == 1]
    df_unambiguous = df[df['unambiguous'] == 1]
    df_weak = df[df['weak'] == 1]
    df_very_weak = df[df['very weak'] == 1]
    df_elevator = df[df['elevator'] == 1]
    df_elevator_not_semi = df[(df['elevator'] == 1) & (df['semi deterministic'] == 0)]

    print(f"! Classification of input automata")
    print(f"!   # empty: {len(df_empty)}")
    print(f"!   # deterministic: {len(df_deterministic)}")
    print(f"!   # deterministic weak: {len(df_deterministic_weak)}")
    print(f"!   # inherently weak: {len(df_inherently_weak)}")
    print(f"!   # semi-deterministic: {len(df_semi_deterministic)}")
    print(f"!   # terminal: {len(df_terminal)}")
    print(f"!   # unambiguous: {len(df_unambiguous)}")
    print(f"!   # weak: {len(df_weak)}")
    print(f"!   # very weak: {len(df_very_weak)}")
    print(f"!   # elevator: {len(df_elevator)}")
    print(f"!   # elevator not semideterministic: {len(df_elevator_not_semi)}")


# remove too easy automata from processing
def filter_non_easy(df):
    # sanitization based on properties of input automata
    #df = df[df['semi deterministic'] == 1]
    df = df[df['semi deterministic'] == 0]
    df = df[df['inherently weak'] == 0]
    df = df[df['unambiguous'] == 0]
    return df


# table to LaTeX file
def table_to_file(table, headers, out_file):
    with open(f"plots/{out_file}.tex", mode='w') as fl:
        print(tab.tabulate(table, headers=headers, tablefmt="latex"), file=fl)

In [11]:
# load results
def load_results(filename):
    df = el.read_file(filename)

    print(f"! Loaded results")
    print(f"!   file:  {filename}")
    print(f"!   time:  {datetime.datetime.now()}")
    print(f"!   # of automata: {len(df)}")
    return df

Then, we set some parameters of the experiments (I know, it would be better if they were in the CSV file... it's on the TODO list).

In [35]:
# in seconds
TIMEOUT = 300
TIMEOUT_VAL = TIMEOUT * 1.1
TIME_MIN = 0.01

FILENAME="results/results-2022-09-08-ltl/from_ltl_red-to300-X.csv"
CLASSIFICATION="results/results-2022-09-08-ltl/from_ltl_classification.csv"

pd.set_option('display.max_columns', 500)

In [36]:
df = load_results(FILENAME)

! Loaded results
!   file:  results/results-2022-09-08-ltl/from_ltl_red-to300-X.csv
!   time:  2025-08-15 17:11:46.015781
!   # of automata: 1721


In [37]:
df = connect_with_classification(df, CLASSIFICATION)
print_classification(df)

! Classification of input automata
!   # empty: 0
!   # deterministic: 555
!   # deterministic weak: 451
!   # inherently weak: 948
!   # semi-deterministic: 1163
!   # terminal: 208
!   # unambiguous: 610
!   # weak: 947
!   # very weak: 777
!   # elevator: 1540
!   # elevator not semideterministic: 377


We should now remove easy automata from the data set

In [39]:
df = filter_non_easy(df)
print(f'! # of automata after sanitization: {len(df)}')
df

! # of automata after sanitization: 414


,name,cola-runtime,cola-States,cola-autfilt-ba-States,cola-autfilt-tgba-States,kofola-334e5d4b-runtime,kofola-334e5d4b-States,kofola-334e5d4b-autfilt-tgba-States,kofola-334e5d4b-autfilt-ba-States,kofola-334e5d4b-merge-runtime,kofola-334e5d4b-merge-States,kofola-334e5d4b-merge-autfilt-tgba-States,kofola-334e5d4b-merge-autfilt-ba-States,kofola-334e5d4b-merge-sim-runtime,kofola-334e5d4b-merge-sim-States,kofola-334e5d4b-merge-sim-autfilt-tgba-States,kofola-334e5d4b-merge-sim-autfilt-ba-States,kofola-334e5d4b-scc-comp-runtime,kofola-334e5d4b-scc-comp-States,kofola-334e5d4b-scc-comp-autfilt-tgba-States,kofola-334e5d4b-scc-comp-autfilt-ba-States,kofola-334e5d4b-scc-comp-high-runtime,kofola-334e5d4b-scc-comp-high-States,kofola-334e5d4b-scc-comp-high-autfilt-tgba-States,kofola-334e5d4b-scc-comp-high-autfilt-ba-States,empty,deterministic,inherently weak,semi deterministic,terminal,unambiguous,weak,very weak,elevator,Unnamed: 10
40,automata/from_ltl/literature_det_red/135.hoa,1.96,33,33,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN
152,automata/from_ltl/literature_nd_red/1.hoa,0.08,13,19,13,1.99,97.0,97.0,119.0,0.80,41.0,41.0,54.0,0.50,41.0,41.0,54.0,0.18,37.0,37.0,31.0,0.19,37.0,37.0,31.0,0,0,0,0,0,0,0,0,1,NaN
153,automata/from_ltl/literature_nd_red/10.hoa,0.06,10,10,10,0.28,16.0,12.0,12.0,0.29,16.0,12.0,12.0,0.29,16.0,12.0,12.0,0.28,16.0,12.0,12.0,0.26,12.0,12.0,12.0,0,0,0,0,0,0,0,0,0,NaN
154,automata/from_ltl/literature_nd_red/11.hoa,0.09,16,17,16,0.65,27.0,27.0,28.0,0.52,27.0,27.0,28.0,0.63,27.0,27.0,28.0,0.50,26.0,22.0,22.0,0.51,25.0,22.0,18.0,0,0,0,0,0,0,0,0,0,NaN
157,automata/from_ltl/literature_nd_red/14.hoa,123.13,50,50,50,49.48,82.0,50.0,50.0,50.04,82.0,50.0,50.0,49.99,82.0,50.0,50.0,49.67,82.0,50.0,50.0,36.83,50.0,50.0,50.0,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,automata/from_ltl/random_nd_red/95.hoa,0.03,5,5,5,0.05,4.0,4.0,4.0,0.07,4.0,4.0,4.0,0.05,4.0,4.0,4.0,0.05,4.0,4.0,4.0,0.06,4.0,4.0,4.0,0,0,0,0,0,0,0,0,1,NaN
1217,automata/from_ltl/random_nd_red/96.hoa,0.04,9,9,9,0.04,8.0,8.0,8.0,0.07,8.0,8.0,8.0,0.06,8.0,8.0,8.0,0.05,8.0,8.0,8.0,0.05,7.0,7.0,8.0,0,0,0,0,0,0,0,0,0,NaN
1218,automata/from_ltl/random_nd_red/97.hoa,0.04,13,13,13,0.10,12.0,11.0,11.0,0.10,12.0,11.0,11.0,0.09,12.0,11.0,11.0,0.07,11.0,9.0,9.0,0.07,9.0,9.0,9.0,0,0,0,0,0,0,0,0,1,NaN
1219,automata/from_ltl/random_nd_red/98.hoa,0.03,12,13,12,0.15,32.0,32.0,33.0,0.09,17.0,17.0,18.0,0.09,17.0,17.0,18.0,0.06,12.0,12.0,13.0,0.08,12.0,12.0,13.0,0,0,0,0,0,0,0,0,1,NaN


Let us now compute some summary statistics

In [41]:
summary_states = dict()
for col in df.columns:
    if re.search('-States$', col) or re.search('-runtime$', col):
        summary_states[col] = dict()
        summary_states[col]['max'] = df[col].max()
        summary_states[col]['min'] = df[col].min()
        summary_states[col]['mean'] = df[col].mean()
        summary_states[col]['median'] = df[col].median()
        summary_states[col]['std'] = df[col].std()
        summary_states[col]['timeouts'] = df[col].isna().sum()

df_summary_states = pd.DataFrame(summary_states).transpose()

################  states of complements ##################
interesting = ["cola",
               "kofola-334e5d4b"
              ]

df_summary_states.loc[[x + '-States' for x in interesting]]

,max,min,mean,median,std,timeouts
cola-States,95.0,2.0,11.550725,9.0,10.439777,0.0
kofola-334e5d4b-States,246.0,2.0,22.078086,13.0,29.260178,17.0


Let us now evaluate the runtime

In [42]:
interesting = [x for x in interesting if not 'nopost' in x]
interesting = [x.replace('-autfilt', '') for x in interesting]
df_summary_states.loc[[x + '-runtime' for x in interesting]]

,max,min,mean,median,std,timeouts
cola-runtime,123.13,0.01,0.350507,0.04,6.049740,0.0
kofola-334e5d4b-runtime,286.73,0.04,2.048338,0.08,15.704322,17.0


Let us now sanitize timeouts and other fishy values in order to have better plots.

In [26]:
 # min and max states
states_min = 1
states_max = df_summary_states['max'].max()
states_timeout = states_max * 1.1

# sanitizing NAs
for col in df.columns:
    if re.search('-States$', col):
        df[col].fillna(states_timeout, inplace=True)
        df[col].replace(0, states_min, inplace=True)  # to remove 0 (in case of log graph)

    if re.search('-runtime$', col):
        df[col].fillna(TIMEOUT_VAL, inplace=True)
        df.loc[df[col] < TIME_MIN, col] = TIME_MIN  # to remove 0 (in case of log graph)
        
############# the best solution out of other tools ##########
df['other_min-States'] = df[['safra-autfilt-States',
                             'piterman-autfilt-States',
                             'spot-autfilt-States',
                             'fribourg-autfilt-States',
                             'ltl2dstar-autfilt-States',
                             'seminator-autfilt-States',
                             'roll-autfilt-States']].min(axis=1)

/var/folders/k8/vb71jppj43x35pc3qvksjy4w0000gn/T/ipykernel_35126/4076596835.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


/var/folders/k8/vb71jppj43x35pc3qvksjy4w0000gn/T/ipykernel_35126/4076596835.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

KeyError: "None of [Index(['safra-autfilt-States', 'piterman-autfilt-States',\n       'spot-autfilt-States', 'fribourg-autfilt-States',\n       'ltl2dstar-autfilt-States', 'seminator-autfilt-States',\n       'roll-autfilt-States'],\n      dtype='object')] are in the [columns]"

And let's draw some scatter plots!

In [ ]:
def scatplot(params):
    size = 8
    if 'xname' not in params:
        params['xname'] = None
    if 'yname' not in params:
        params['yname'] = None
    if 'max' not in params:
        params['max'] = 10000
    if 'tickCount' not in params:
        params['tickCount'] = 5
    if 'filename' not in params:
        params['filename'] = "fig_" + params['x'] + "_vs_" + params['y']

    pl = el.scatter_plot(df,
                         xcol=params['x'] + '-States',
                         ycol=params['y'] + '-States',
                         xname=params['xname'], yname=params['yname'],
                         domain=[states_min, params['max']],
                         tickCount=params['tickCount'],
                         log=True, width=size, height=size)
    return pl

In [ ]:
display(scatplot({'x': 'ranker-maxr-nopost' , 'y': 'ranker-rrestr-nopost', 'max': states_timeout}))
display(scatplot({'x': 'ranker-maxr-nopost' , 'y': 'schewe', 'max': states_timeout}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "seminator-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "piterman-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "fribourg-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "roll-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "ltl2dstar-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "spot-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "safra-autfilt"}))
display(scatplot({'x': "safra-autfilt"      , 'y': "spot-autfilt"}))
display(scatplot({'x': "piterman-autfilt"   , 'y': "spot-autfilt"}))
display(scatplot({'x': "piterman-autfilt"   , 'y': "ltl2dstar-autfilt"}))
display(scatplot({'x': "piterman-autfilt"   , 'y': "seminator-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "other_min"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "ranker-maxr-delayopt-autfilt"}))
display(scatplot({'x': "ranker-maxr-autfilt", 'y': "ranker-maxr-delayopt-new-autfilt"}))
display(scatplot({'x': "ranker-maxr-delayopt-autfilt", 'y': "ranker-maxr-delayopt-new-autfilt"}))

In [ ]:
# generate evaluation
def gen_evaluation(file_input, file_classification, out_prefix):
   

    # headers = ["name", "min", "max", "mean", "median", "std. dev", "timeouts"]
    headers = ["method", "max", "mean", "median", "std. dev", "timeouts"]
    print("######################################################################")
    print("####                        Table 1 (left)                        ####")
    print("######################################################################")
    print(tab.tabulate(tab_interesting, headers=headers, tablefmt="github"))
    table_to_file(tab_interesting, headers, out_prefix + "_table1left")
    print("\n\n")

   
    headers = ["method", "mean", "median", "std. dev"]
    # headers = ["method", "min", "max", "mean", "median", "std. dev", "timeouts"]
    print("######################################################################")
    print("####                           Table 2                            ####")
    print("######################################################################")
    print(tab.tabulate(tab_interesting, headers=headers, tablefmt="github"))
    table_to_file(tab_interesting, headers, out_prefix + "_table2")
    print("\n\n")

    # comparing wins/loses
    compare_methods = [("ranker-maxr-nopost-States", "ranker-rrestr-nopost-States"),
                       ("ranker-maxr-nopost-States", "schewe-States"),
                       ("ranker-maxr-autfilt-States", "piterman-autfilt-States"),
                       ("ranker-maxr-autfilt-States", "safra-autfilt-States"),
                       ("ranker-maxr-autfilt-States", "spot-autfilt-States"),
                       ("ranker-maxr-autfilt-States", "fribourg-autfilt-States"),
                       ("ranker-maxr-autfilt-States", "ltl2dstar-autfilt-States"),
                       ("ranker-maxr-autfilt-States", "seminator-autfilt-States"),
                       ("ranker-maxr-autfilt-States", "roll-autfilt-States"),

                      ]
 #   compare_methods = [("ranker-maxr-nopost-States", "ranker-rrestr-nopost-States"),
 #                      ("ranker-maxr-nopost-States", "schewe-States"),
 #                      ("ranker-maxr-bo-autfilt-States", "piterman-autfilt-States"),
 #                      ("ranker-maxr-bo-autfilt-States", "safra-autfilt-States"),
 #                      ("ranker-maxr-bo-autfilt-States", "spot-autfilt-States"),
 #                      ("ranker-maxr-bo-autfilt-States", "fribourg-autfilt-States"),
 #                      ("ranker-maxr-bo-autfilt-States", "ltl2dstar-autfilt-States"),
 #                      ("ranker-maxr-bo-autfilt-States", "seminator-autfilt-States"),
 #                      ("ranker-maxr-bo-autfilt-States", "roll-autfilt-States"),
 #                      ]

    tab_wins = []
    for left, right in compare_methods  :
        left_over_right = df[df[left] < df[right]]
        right_timeouts = left_over_right[left_over_right[right] == states_timeout]

        right_over_left = df[df[left] > df[right]]
        left_timeouts = right_over_left[right_over_left[left] == states_timeout]

        tab_wins.append([right, len(left_over_right), len(right_timeouts), len(right_over_left), len(left_timeouts)])

    headers_wins = ["method", "wins", "wins-timeouts", "loses", "loses-timeouts"]
    print("######################################################################")
    print("####                        Table 1 (right)                       ####")
    print("######################################################################")
    print(tab.tabulate(tab_wins, headers=headers_wins, tablefmt="github"))
    table_to_file(tab_wins, headers_wins, out_prefix + "_table1right")
    print("\n\n")

    print("##############    other claimed results    ###############")

    ############# the best solution ##########
    df['other_min-States'] = df[
        ['safra-autfilt-States', 'piterman-autfilt-States',
         'spot-autfilt-States', 'fribourg-autfilt-States',
         'ltl2dstar-autfilt-States','seminator-autfilt-States',
         'roll-autfilt-States']].min(axis=1)

    ranker_best = df[df['ranker-maxr-autfilt-States'] < df['other_min-States']]
    ranker_not_best = df[df['ranker-maxr-autfilt-States'] > df['other_min-States']]

    num_ranker_not_strictly_best = len(df) - len(ranker_not_best)
    num_ranker_not_strictly_best_percent = "{:.1f}".format(num_ranker_not_strictly_best / len(df) * 100)
    num_ranker_strictly_best = len(ranker_best)
    num_ranker_strictly_best_percent = "{:.1f}".format(num_ranker_strictly_best / len(df) * 100)
    print(f"ranker non-strictly best: {num_ranker_not_strictly_best} (= {num_ranker_not_strictly_best_percent} %)")
    print(f"ranker stricly best: {num_ranker_strictly_best} (= {num_ranker_strictly_best_percent} %)")
    # print(f"ranker not best = {len(ranker_not_best)}")

    ###########   BackOff   ################
    backoff = df[df["ranker-maxr-bo-Engine"].str.contains("GOAL", na=False)]
    print(f"backoff executions: {len(backoff)}")

    to_cmp2 = [{'x': "ranker-maxr-nopost", 'y': "ranker-rrestr-nopost",
                'xname': 'Ranker-MaxR', 'yname': 'Ranker-RRestr'},
               {'x': "ranker-maxr-nopost", 'y': "schewe",
                'xname': "Ranker-MaxR", 'yname': "Schewe-RedAvgOut"},
               {'x': "ranker-maxr-autfilt", 'y': "seminator-autfilt",
                'xname': "Ranker-MaxR+PP", 'yname': "Seminator 2+PP",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "piterman-autfilt",
                'xname': "Ranker-MaxR+PP", 'yname': "Piterman+PP",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "fribourg-autfilt",
                'xname': "Ranker-MaxR+PP", 'yname': "Fribourg+PP",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "roll-autfilt",
                'xname': "Ranker-MaxR+PP", 'yname': "ROLL+PP",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "ltl2dstar-autfilt",
                'xname': "Ranker-MaxR+PP", 'yname': "ltl2dstar+PP",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "spot-autfilt",
                'xname': "Ranker-MaxR+PP", 'yname': "Spot+PP",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "safra-autfilt",
                'xname': "Ranker-MaxR+PP", 'yname': "Safra+PP",
                'max': 10000, 'tickCount': 3},
               {'x': "safra-autfilt", 'y': "spot-autfilt",
                'max': 10000, 'tickCount': 3},
               {'x': "piterman-autfilt", 'y': "spot-autfilt",
                'max': 10000, 'tickCount': 3},
               {'x': "piterman-autfilt", 'y': "ltl2dstar-autfilt",
                'max': 10000, 'tickCount': 3},
               {'x': "piterman-autfilt", 'y': "seminator-autfilt",
                'max': 10000, 'tickCount': 3},
      #         {'x': "piterman-gff-autfilt", 'y': "seminator-autfilt",
      #          'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "other_min",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "ranker-maxr-delayopt-autfilt",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-autfilt", 'y': "ranker-maxr-delayopt-new-autfilt",
                'max': 10000, 'tickCount': 3},
               {'x': "ranker-maxr-delayopt-autfilt", 'y': "ranker-maxr-delayopt-new-autfilt",
                'max': 10000, 'tickCount': 3},
              ]

    # add fields where not present
    for params in to_cmp2:
        if 'xname' not in params:
            params['xname'] = None
        if 'yname' not in params:
            params['yname'] = None
        if 'max' not in params:
            params['max'] = states_timeout
        if 'tickCount' not in params:
            params['tickCount'] = 5
        if 'filename' not in params:
            params['filename'] = "fig_" + params['x'] + "_vs_" + params['y']

    size = 8
    plot_list = [(params['x'],
                  params['y'],
                  params['filename'],
                  el.scatter_plot(df,
                               xcol=params['x'] + '-States',
                               ycol=params['y'] + '-States',
                               xname=params['xname'], yname=params['yname'],
                               domain=[states_min, params['max']],
                               tickCount=params['tickCount'],
                               log=True, width=size, height=size)) for params
                 in to_cmp2]

    print("\n\n")
    print("Generating plots...")
    for x, y, filename, plot in plot_list:
        filename = f"plots/{out_prefix}_{filename}.pdf"
        print(f"plotting x: {x}, y: {y}... saving to {filename}")
        # plot.save(filename, scale_factor=2)
        plot.save(filename=filename, dpi=1000)
        print(plot)

    # return benchmarks solvable only by 'engine'
    def only_solves(df, engine):
        # select those where engine finishes
        res = df[df[engine + '-States'] != states_timeout]
        for col in res.columns:
            if re.search('-States$', col) and not re.search(engine, col):
                res = res[res[col] == states_timeout]

        return res


    engines = ["ranker-maxr",
               "ranker-rrestr",
               "ranker-maxr-bo",
               "piterman",
               "safra",
               "spot",
               "fribourg",
               "ltl2dstar",
               "seminator",
               "roll",
              ]

    for i in engines:
        i_only_solves = only_solves(df, i)
        print(f"only {i} = " + str(len(i_only_solves)))
        if len(i_only_solves) > 0:
            print()
            print(tab.tabulate(i_only_solves, headers='keys'))
            print()

    def none_solves(df):
        # select those where engine finishes
        res = df
        for col in res.columns:
            if re.search('-States$', col):
                res = res[res[col] == states_timeout]

        return res

    unsolvable = none_solves(df)
    print("unsolvable: " + str(len(unsolvable)))
    print(tab.tabulate(unsolvable, headers='keys'))
    print("\n\n\n\n\n")

    # comparing delayopt on only terminating cases

    df_delay = df[(df['ranker-maxr-delayopt-runtime'] != TIMEOUT_VAL) &
                  (df['ranker-maxr-delayopt-new-runtime'] != TIMEOUT_VAL) &
                  (df['ranker-maxr-runtime'] != TIMEOUT_VAL)]

    print(df_delay)

    summary_states = dict()
    for col in df_delay.columns:
        if re.search('-States$', col) or re.search('-runtime$', col):
            summary_states[col] = dict()
            summary_states[col]['max'] = df_delay[col].max()
            summary_states[col]['min'] = df_delay[col].min()
            summary_states[col]['mean'] = df_delay[col].mean()
            summary_states[col]['median'] = df_delay[col].median()
            summary_states[col]['std'] = df_delay[col].std()
            summary_states[col]['timeouts'] = len(df_delay[df_delay[col] == states_timeout])

    df_summary_states = pd.DataFrame(summary_states).transpose()

    print(tab.tabulate(df_summary_states, headers='keys', tablefmt="github"))

   # df_piterman_gff = df[(df['piterman-gff-autfilt-States'] == 1) & (df['seminator-autfilt-States'] > 500)]
   # print(tab.tabulate(df_piterman_gff, headers='keys'))

#############################################################################
############################## ENTRY POINT ##################################
#############################################################################

#gen_evaluation("results/results-2021-01-21/random-all-to300-new-22jan21.csv",
#               "random-all-compact-classification.csv",
#               "random-all")
#
#gen_evaluation("results/results-2021-01-04/from_ltl_red-to300-new-jan21.csv",
#               "from_ltl_classification.csv",
#               "from_ltl")

#gen_evaluation("results/results-2021-01-04/pokus.csv",
#               "random-all-classification.csv",
#               "gff")

gen_evaluation("results/results-2021-05-28-delayopt-new/random-all-to300-plus-ranker-delayopt-new.csv",
               "random-all-compact-classification.csv",
               "random-all")

#gen_evaluation("results/results-2021-05-19-roll/random-all-to300-roll-new-may19-connected.csv",
#               "random-all-compact-classification.csv",
#               "random-all")

In [ ]:
print("ahoj")